### Что хочу сделать

1. Разбить на предложения, составить вектор частотности длин предложений нормированный к общему числу предложений в тексте (+ график распределения длин)
2. Посчитать общее и среднее число знаков препинания в предложениях с учетом длины предложения "пунктуационный профиль"
3. По каждому предложению сделать грамматический разбор (части речи с родами/падежами там где они применимы) посчитать частотность по частям речи нормированную на длину предложения) 
4. Посчитать "любимый род" и "любимый падеж"
5. Подсчитать N (100) любимых существительных и глаголов

In [1]:
import numpy as np
import pandas as pd
import nltk
import pickle
import re

from pymystem3 import Mystem
from matplotlib import pyplot as plt

tok = nltk.data.load('tokenizers/punkt/russian.pickle')
mystem_analyzer = Mystem()

In [2]:
def extract_morph(stem):
    structure = [] # только части речи
    structure_full = [] # полностью вся грамматическая форма
    for lexem in stem:
        try:
            if 'analysis' in lexem.keys() and lexem['analysis'] != []:
                
                #есть спорные лексемы - они идут массивом, но в части речи сомнения как правило нет
                #if len(lexem['analysis']) >= 2:
                #    print(lexem['analysis'])
                
                part = lexem['analysis'][0]['gr']
                structure_full.append(part)
                structure.append(re.match(r'\w*',part).group(0))
        except:
            print("Error:",lexem)
    return structure, structure_full

In [3]:
source_df = pd.read_csv("raw_data/found_df.csv")
files = source_df["file_name"]

for file_name in files:
   
    file = "raw_data/" + file_name + ".txt"
    print("File: ", file_name)
    
    with open(file,"r") as fp:
        string = fp.read()[12:]
        # нужно побороть артифакты вида 'дышать.В' (после точки должен стоять пробел)
        # а также несколько подряд идущих пробелов
    string = re.sub(r'\s+|\n', r' ' , string)    
    string = re.sub(r'(\w)([\.:])(\w)', r'\1\2 \3' , string)    
    
    sentences = tok.tokenize(string)
    # исключил не используемые в худ текстах, они - шум 
    punct = '"!\'()*,-.:;?' # 12 шт

    # считаем пунктуацию
    punct_arr = []
    for sentence in sentences:
        punct_t = []
        for p in punct:
            punct_t.append(sentence.count(p))
        punct_arr.append(punct_t)  
    
    # теперь убираем пунктуацию и считаем длины предложений с учетом иностранных слов
    n_words = []
    for sentence in sentences:
        n_words.append(len(sentence.split()))

    # хочется зачистить от иностранщины и артифактов. оставить только русские символы и пунктуацию. 
    # конечно при этом будут битые предложения но без вариантов. На регулярках убиваю все символы кроме русских 
    # пунктуацию мы уже посчитали, она больше не нужна

    sent_clean = []
    for sentence in sentences:
        sentence = re.sub(r"[^а-яА-ЯёЁ \-]", '', sentence) # нужно оставить "что-то"
        sentence = re.sub(r"\s?-\s|\s-\s?", ' ', sentence) # и убрать тире отдельное 
        sent_clean.append(sentence)

    # считаем число слов в предложениях
    morph_arr = []
    for sentence in sent_clean:
        morph = mystem_analyzer.analyze(sentence)
        short, full = extract_morph(morph)
        morph_arr.append([short, full, morph])        

    res_arr = [sentences, n_words, punct_arr, morph_arr]  
    with open("./stats-data/" + file_name + ".pkl","wb") as fp:
        pickle.dump(res_arr, fp)    

File:  aleksandr_pushkin_arap_petra_velikogo
File:  aleksandr_pushkin_dubrovskij
File:  aleksandr_pushkin_istorija_sela_gorjuhina
File:  aleksandr_pushkin_kapitanskaja_dochka
File:  aleksandr_pushkin_pikovaja_dama
File:  aleksandr_pushkin_povesti_pokojnogo_ivana_petrovicha_belkina
File:  aleksandr_pushkin_puteshestvie_v_arzrum_vo_vremja_pohoda_1829_goda
File:  dmitrij_mamin-sibirjak_ak-bozat
File:  ivan_turgenev_andrej_kolosov
File:  dmitrij_mamin-sibirjak_bogach_i_eremka
File:  dmitrij_mamin-sibirjak_v_glushi
File:  dmitrij_mamin-sibirjak_vertel
File:  dmitrij_mamin-sibirjak_zimov'e_na_studenoj
File:  dmitrij_mamin-sibirjak_postojko
File:  dmitrij_mamin-sibirjak_priemysh
File:  dmitrij_mamin-sibirjak_seraja_shejka
File:  dmitrij_mamin-sibirjak_staryj_vorobej
File:  ivan_turgenev_asja
File:  ivan_turgenev_zapiski_ohotnika
File:  ivan_turgenev_mumu
File:  ivan_turgenev_ottsy_i_deti
File:  ivan_turgenev_rudin
File:  anton_chehov_dvadtsat'_devjatoe_ijunja
File:  anton_chehov_kryzhovnik
Fi

### Проверка

In [4]:
with open("./stats-data/" + "fedor_dostoevskij_bednye_ljudi.pkl", "rb") as fp:
    c = pickle.load(fp)

In [5]:
c[0][0],c[1][0],c[2][0],c[3][0][0],

('Ох уж эти мне сказочники!',
 5,
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 ['INTJ', 'PART', 'APRO', 'SPRO', 'S'])

### Общие массивы по всем авторам 

In [6]:
source_df.head()

,Unnamed: 0,title,author,author_id,file_name,text_url
0,0,Арап Петра Великого,Александр Пушкин,0,aleksandr_pushkin_arap_petra_velikogo,https://ilibrary.ru/text/476/p.1/index.html
1,1,Дубровский,Александр Пушкин,0,aleksandr_pushkin_dubrovskij,https://ilibrary.ru/text/479/p.1/index.html
2,2,История села Горюхина,Александр Пушкин,0,aleksandr_pushkin_istorija_sela_gorjuhina,https://ilibrary.ru/text/477/p.1/index.html
3,3,Капитанская дочка,Александр Пушкин,0,aleksandr_pushkin_kapitanskaja_dochka,https://ilibrary.ru/text/107/p.1/index.html
4,4,Пиковая дама,Александр Пушкин,0,aleksandr_pushkin_pikovaja_dama,https://ilibrary.ru/text/480/p.1/index.html


In [7]:
stats_dict = {}
authos_ids = source_df["author_id"].unique()
for author_id in authos_ids:
    files = source_df[source_df["author_id"]==author_id]["file_name"]
    stats = {"sentences": [], "sentence_length": [], "punctuation" : [], "gram": []}
    for file in files:
        with open("./stats-data/" + file + ".pkl", "rb") as fp:
            data_arr = pickle.load(fp)        
        stats["sentences"].extend(data_arr[0])
        stats["sentence_length"].extend(data_arr[1])
        stats["punctuation"].extend(data_arr[2])
        stats["gram"].extend(data_arr[3])
    with open("./stats-data/" + str(author_id) + ".pkl","wb") as fp:
        pickle.dump(stats, fp)   
             

In [8]:
source_df[["author", "author_id"]].groupby("author_id").max()

,author
author_id,
0,Александр Пушкин
1,Дмитрий Мамин-Сибиряк
2,Иван Тургенев
3,Антон Чехов
4,Николай Гоголь
5,Иван Бунин
6,Александр Куприн
7,Андрей Платонов
8,Всеволод Гаршин


In [9]:
with open("./stats-data/4.pkl", "rb") as fp:
    test_arr = pickle.load(fp)
    for key, item in test_arr.items():
        print(key, ": ", str(len(item)), " записей")

sentences :  5946  записей
sentence_length :  5946  записей
punctuation :  5946  записей
gram :  5946  записей
